*Run script to set up environment pip install packages*

In [ ]:
# !pip install --upgrade azureml-sdk azureml-widgets

# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

## Initialise the workspace
have to initialise first in order to get and upload the data if not already created

In [2]:
ws = Workspace.from_config()

## Dataset

In [3]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False


key = "job-leaver-aug-small"
local_path = './data_blob_upload/job_leaver_aug_small.csv'
target_path = "job-leaver-aug-small-automl-data"

description_text = "This is an extract from the original Kaggle source: https://www.kaggle.com/arashnic/hr-analytics-job-change-of-data-scientists."

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        print("dataset found")

if not found:
        # I have changed the import method because uploading the csv conventionally created some issues
        # Importing into pandas dataframe and then uploading the dataset stopped the previous import issues
        
        # upload to pandas dataframe
        df = pd.read_csv(local_path)
        
        # get the datastore to upload prepared data
        datastore = ws.get_default_datastore()

        # upload the local file from src_dir to the target_path in datastore
        dataset = Dataset.Tabular.register_pandas_dataframe(name=key, dataframe=df, target=(datastore, target_path), show_progress=True)

        # create a dataset referencing the cloud location
        dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, (target_path))], encoding='utf8', header=True)

        dataset = dataset.register(
                workspace=ws,
                name=key,
                description=description_text)
        print("dataset uploaded and registered")

dataset found


## Experiment
set up the experiment

In [4]:
experiment_name = 'data-science-job-leavers-small-automl'
project_folder = './project_automl'

experiment=Experiment(ws, experiment_name)

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

amlcompute_cluster_name = "standard-ds12-v2"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           vm_priority = 'lowpriority',
                                                           min_nodes=0,
                                                           max_nodes=5)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
automl_settings = {
    "experiment_timeout_minutes": 15,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="target",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
from azureml.widgets import RunDetails

# Show the status in the notebook as the experiment runs
RunDetails(remote_run).show()
remote_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_0d1cb3c5-44c5-4bee-81a5-a3646969425a',
 'target': 'standard-ds12-v2',
 'status': 'Completed',
 'startTimeUtc': '2021-01-02T18:09:30.346584Z',
 'endTimeUtc': '2021-01-02T18:26:48.394775Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'standard-ds12-v2',
  'AMLSettingsJsonString': '{"path":null,"name":"data-science-job-leavers-small-automl","subscription_id":"ea9f71e6-aef9-4b3a-837d-f651463b4205","resource_group":"WS2020120214_rg","workspace_name":"WS2020120214","region":"uksouth","compute_target":"standard-ds12-v2","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max"

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
best_run, fitted_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

balanced_accuracy 0.6434460026560024
matthews_correlation 0.33913409028624436
AUC_weighted 0.7866389250101649
recall_score_weighted 0.7900025774277272
f1_score_macro 0.6589172624699359
accuracy 0.7900025774277272
recall_score_macro 0.6434460026560024
average_precision_score_micro 0.8540659425715663
norm_macro_recall 0.2868920053120047
average_precision_score_macro 0.7259979274421097
precision_score_macro 0.7012469699759146
AUC_micro 0.876960698574282
AUC_macro 0.7866389250101649
precision_score_weighted 0.7713967946099544
average_precision_score_weighted 0.8170693121513467
recall_score_micro 0.7900025774277272
f1_score_weighted 0.7730757305347463
f1_score_micro 0.7900025774277273
precision_score_micro 0.7900025774277272
weighted_accuracy 0.8701107131528808
log_loss 0.46104432271721735
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_0d1cb3c5-44c5-4bee-81a5-a3646969425a_38/accuracy_table
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_0d1cb3c5-44c5-4bee-81a5-a364696

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [6]:
# # I closed the notebook and used this script to allow me 
# # to continue the work without rerunning all pf the previous code
# # <DEV>
# run_id = "AutoML_0d1cb3c5-44c5-4bee-81a5-a3646969425a"
# remote_run = ws.get_run(run_id)
# run = remote_run
# best_run, fitted_model = remote_run.get_output()
# # </DEV>

In [7]:
run = remote_run
model_name = best_run.properties["model_name"]
model = run.register_model(model_name=model_name, description="job-leaver automl", tags=None)

In [8]:
import os

folder_name = 'job_leaver_service'

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(experiment_folder, exist_ok=True)

print(folder_name, 'folder created.')

job_leaver_service folder created.


In [9]:
from azureml.core import Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.automl.core.shared import constants

In [10]:
folder_name = 'job_leaver_service'
entry_script = os.path.join(folder_name, "score.py")
env_file = os.path.join(folder_name, "env.yml")
best_run.download_file("outputs/scoring_file_v_1_0_0.py", entry_script)
best_run.download_file(constants.CONDA_ENV_FILE_PATH, env_file)

In [11]:
env = Environment.from_conda_specification(name="env", file_path=env_file)
inference_config = InferenceConfig(entry_script=entry_script, environment=env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, description="job-leavers classification")

service = Model.deploy(ws, 
                       name="job-leaver-service", 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aci_config)

service.wait_for_deployment(True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.....................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [12]:
import re
import pandas as pd
import json
import requests


# get the associated scoring_uri from the workspace
# there must be a better way but ran out of time to find it
text = str(ws.webservices['job-leaver-service'])

try:
    scoring_uri = re.search('tags=(http://[^,]+)', text).group(1)
except AttributeError:
    scoring_uri = None

scoring_uri

# get sample data by importing the first record of the kaggle test file and outputting it as a json object

data = {"data": [{k: [i for i in v.values()][0] for (k,v) in pd.read_csv('data_archive/aug_test.csv', nrows=1).to_dict().items()}]}

# check if the scoring_uri has been found and submit a request to it, 
# using the converted data extracted fromt the kaggle sample data 

if scoring_uri:
    # Convert to JSON string
    input_data = json.dumps(data)
    with open("data.json", "w") as _f:
        _f.write(input_data)

    # Set the content type
    headers = {"Content-Type": "application/json"}

    # Make the request and display the response
    resp = requests.post(scoring_uri, input_data, headers=headers)
    print(resp.json())

{"result": [0.0]}


TODO: In the cell below, print the logs of the web service and delete the service

In [13]:
print(service.get_logs())

2021-01-04T16:26:15,504635300+00:00 - iot-server/run 
2021-01-04T16:26:15,511290600+00:00 - nginx/run 
2021-01-04T16:26:15,513268800+00:00 - gunicorn/run 
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-01-04T16:26:15,529706200+00:00 - rsyslog/run 
rsyslogd

In [14]:
service.delete()
model.delete()